# Predictions Sales

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score

pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)  
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings("ignore")

In [5]:
def extract_color(product_description):
    """
    Extracts the color from a product description.
    
    It assumes that the color appears after an 8-digit number.
    Example:
      Input: 'Via Uno Botin 12231407 Marron'
      Output: 'Marron'
    """
    # Pattern explanation:
    # \d{8}  : Match exactly 8 digits.
    # \s+    : Match one or more whitespace characters.
    # (.*)$  : Capture everything until the end of the string (expected to be the color).
    pattern = r'\d{8}\s+(.*)$'
    match = re.search(pattern, product_description)
    if match:
        return match.group(1).strip()
    else:
        return None

In [6]:
# Load the data
data = pd.read_excel('/home/sebastian/Documents/Portfolio/Ventas via uno/Database/principal.xlsx')
test_data = pd.read_excel("/home/sebastian/Downloads/costo venta ene 2025.xlsx")
data.columns

Index(['Tipo Movimiento', 'Tipo Documento', 'Número Documento',
       'Fecha de Emisión', 'Tracking number', 'Fecha y Hora Venta', 'Sucursal',
       'Vendedor', 'Cliente Nombre', 'Cliente RUT', 'Cliente Email',
       'Cliente Dirección', 'Cliente Comuna', 'Cliente Ciudad',
       'Lista de precio', 'Tipo de entrega', 'Moneda',
       'Tipo de Producto/Servicio', 'SKU', 'Producto/Servicio', 'Variante',
       'Otros Atributos', 'Marca', 'Detalle de Productos/Servicios Pack/Promo',
       'Precio base', 'Precio Neto Unitario', 'Precio Bruto Unitario',
       'Cantidad', 'Subtotal Neto', 'Subtotal Impuestos', 'Subtotal Bruto',
       'Nombre de dcto', 'Descuento Neto', 'Descuento Bruto', '% Descuento',
       'Costo neto unitario', 'Costo Total Neto', 'Margen', '% Margen',
       'Producto', 'Marketplace'],
      dtype='object')

In [7]:
# Verify null values
data.info()
display(data.isnull().sum())

# Drop columns with null values and columns that are not useful
data.drop(['Marca', 'Otros Atributos', 'Fecha de Emisión'], axis=1, inplace=True)

# Split Fecha y Hora Venta in month, day and hour.
data['Month'] = data['Fecha y Hora Venta'].dt.month
data['Day'] = data['Fecha y Hora Venta'].dt.day
data['Hour'] = data['Fecha y Hora Venta'].dt.hour
data.drop('Fecha y Hora Venta', axis=1, inplace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35438 entries, 0 to 35437
Data columns (total 41 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   Tipo Movimiento                            35438 non-null  object        
 1   Tipo Documento                             35438 non-null  object        
 2   Número Documento                           35438 non-null  int64         
 3   Fecha de Emisión                           15407 non-null  datetime64[ns]
 4   Tracking number                            35438 non-null  object        
 5   Fecha y Hora Venta                         35438 non-null  datetime64[ns]
 6   Sucursal                                   35438 non-null  object        
 7   Vendedor                                   35438 non-null  object        
 8   Cliente Nombre                             35438 non-null  object        
 9   Cliente RUT      

Tipo Movimiento                                  0
Tipo Documento                                   0
Número Documento                                 0
Fecha de Emisión                             20031
Tracking number                                  0
Fecha y Hora Venta                               0
Sucursal                                         0
Vendedor                                         0
Cliente Nombre                                   0
Cliente RUT                                      0
Cliente Email                                    0
Cliente Dirección                                0
Cliente Comuna                                   0
Cliente Ciudad                                   0
Lista de precio                                  0
Tipo de entrega                                  0
Moneda                                           0
Tipo de Producto/Servicio                        0
SKU                                              0
Producto/Servicio              

In [8]:
data=data[data['Variante']!='Despacho']    
data=data[data['Variante']!='envio']
data=data[data['Tipo de Producto/Servicio']!='Sin Tipo']
data.columns

Index(['Tipo Movimiento', 'Tipo Documento', 'Número Documento',
       'Tracking number', 'Sucursal', 'Vendedor', 'Cliente Nombre',
       'Cliente RUT', 'Cliente Email', 'Cliente Dirección', 'Cliente Comuna',
       'Cliente Ciudad', 'Lista de precio', 'Tipo de entrega', 'Moneda',
       'Tipo de Producto/Servicio', 'SKU', 'Producto/Servicio', 'Variante',
       'Detalle de Productos/Servicios Pack/Promo', 'Precio base',
       'Precio Neto Unitario', 'Precio Bruto Unitario', 'Cantidad',
       'Subtotal Neto', 'Subtotal Impuestos', 'Subtotal Bruto',
       'Nombre de dcto', 'Descuento Neto', 'Descuento Bruto', '% Descuento',
       'Costo neto unitario', 'Costo Total Neto', 'Margen', '% Margen',
       'Producto', 'Marketplace', 'Month', 'Day', 'Hour'],
      dtype='object')

In [9]:
data['Cliente Ciudad'].unique()

array(['Santiago', 'Sin datos', 'REGIÓN METROPOLITANA', 'Viña Del Mar',
       'Región Metropolitana', 'STGO', 'santiago', 'CAÑETE', 'SANTIAGO',
       'Colina', 'Región del Libertador General Bernardo O’Higgins (',
       'antofagasta', 'RANCAGUA', 'Linares', 'Región de Valparaíso (V)',
       'Región Metropolitana de Santiago', 'Región de Los Lagos (X)',
       'Parral', 'Región de Antofagasta (II)', 'AYSEN', 'ARAUCO', 'LANCO',
       'OSORNO', 'TRAIGUEN', 'TOME', 'TALAGANTE', 'TALCAHUANO',
       'QUILLOTA', 'LOS ANDES', 'METROPOLITANA',
       "Región del Libertador General Bernardo O'Higgins",
       'Región del Biobío (VIII)', 'CALAMA', 'REGIÓN DE VALPARAÍSO (V)',
       'PUENTE ALTO', 'Copiapó', 'REGIÓN DEL ÑUBLE (XVI)', 'PUERTO MONTT',
       'Región de La Araucanía (IX)', 'FRUTILLAR', 'VALPARAISO',
       'Rancagua', 'Talca', 'xxx', 'Región de Tarapacá (I)',
       'METROPOLITANA DE SANTIAGO'], dtype=object)

In [10]:
data['Cliente Ciudad']= data['Cliente Ciudad'].str.lower()
data['Cliente Ciudad'] = data['Cliente Ciudad'].replace(['metropolitana','santiago'])
data['Cliente Ciudad']= data['Cliente Ciudad'].replace(['region metropolitana','santiago'])


## Prediction of price 

Predict volume of sales

In [11]:
# Extract color 
data['Color'] = data['Producto/Servicio'].apply(extract_color)
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 27133 entries, 0 to 35437
Data columns (total 41 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Tipo Movimiento                            27133 non-null  object 
 1   Tipo Documento                             27133 non-null  object 
 2   Número Documento                           27133 non-null  int64  
 3   Tracking number                            27133 non-null  object 
 4   Sucursal                                   27133 non-null  object 
 5   Vendedor                                   27133 non-null  object 
 6   Cliente Nombre                             27133 non-null  object 
 7   Cliente RUT                                27133 non-null  object 
 8   Cliente Email                              27133 non-null  object 
 9   Cliente Dirección                          27133 non-null  object 
 10  Cliente Comuna             

Tipo Movimiento        Tipo Documento  Número Documento  \
0           venta  boleta electrónica t             21923   
1           venta  boleta electrónica t             21924   
2           venta  boleta electrónica t             21925   
3           venta  boleta electrónica t             21926   
4           venta  boleta electrónica t             21927   

            Tracking number     Sucursal      Vendedor    Cliente Nombre  \
0  659208472266bd053b66938c  Casa Matriz  VENTA ONLINE   viviana salinas   
1  659213862266bd053e66957b  Casa Matriz  VENTA ONLINE     Silvia Psijas   
2  659239f57e379f0505d14b6e  Casa Matriz  VENTA ONLINE  Priscilla Bustos   
3  659239f9cb05a8054ae57022  Casa Matriz  VENTA ONLINE  Priscilla Bustos   
4  65923dd57e379f0505d14ba0  Casa Matriz  VENTA ONLINE  Carolina Montiel   

  Cliente RUT              Cliente Email Cliente Dirección  \
0  13617340-5                  Sin datos         Sin datos   
1   6693321-0                  Sin datos         Sin datos   
2  16753486-4                  Sin datos         Sin datos   
3  16753486-4                  Sin datos         Sin datos   
4  15663772-6  montieliglesias@gmail.com           Caren     

         Cliente Comuna Cliente Ciudad        Lista de precio  \
0  region metropolitana       santiago  Lista de Precios Base   
1  region metropolitana       santiago  Lista de Precios Base   
2  region metropolitana       santiago  Lista de Precios Base   
3  region metropolitana       santiago  Lista de Precios Base   
4  region metropolitana       santiago  Lista de Precios Base   

    Tipo de entrega Moneda Tipo de Producto/Servicio             SKU  \
0  retiro en tienda    CLP                     Botin  12231407015007   
1  retiro en tienda    CLP                     Botin  12231407023007   
2  retiro en tienda    CLP                  Sandalia  22323605015006   
3  retiro en tienda    CLP                  Sandalia  22323605015006   
4  retiro en tienda    CLP                      Bota  12231303015008   

                 Producto/Servicio  Variante  \
0    Via Uno Botin 12231407 Marron  37 Negro   
1    Via Uno Botin 12231407 Marron        37   
2  Via Uno Sandalia 22323605 Negro        36   
3  Via Uno Sandalia 22323605 Negro        36   
4      Via Uno Bota 12231303 Negro        38   

   Detalle de Productos/Servicios Pack/Promo  Precio base  \
0                                          0        10076   
1                                          0        10076   
2                                          0        12597   
3                                          0        12597   
4                                          0        10916   

   Precio Neto Unitario  Precio Bruto Unitario  Cantidad  Subtotal Neto  \
0                 10076                  11990         1          10076   
1                 10076                  11990         1          10076   
2                 12597                  14990         1          12597   
3                 12597                  14990         1          12597   
4                 10916                  12990         1          10916   

   Subtotal Impuestos  Subtotal Bruto Nombre de dcto  Descuento Neto  \
0                1914           11990      Sin datos               0   
1                1914           11990      Sin datos               0   
2                2393           14990      Sin datos               0   
3                2393           14990      Sin datos               0   
4                2074           12990      Sin datos               0   

   Descuento Bruto  % Descuento  Costo neto unitario  Costo Total Neto  \
0                0            0               -13098            -13098   
1                0            0               -13098            -13098   
2                0            0                -4500             -4500   
3                0            0                -4500             -4500   
4                0            0               -14222            -14222 

In [12]:
import lightgbm as lgb
from sklearn.preprocessing import TargetEncoder
data['Month_sin'] = np.sin(2 * np.pi * data['Month']/12)
data['Month_cos'] = np.cos(2 * np.pi * data['Month']/12)

# Predict volume of sales 
Feautures = ['Month_sin', 'Month_cos','Marketplace', 'Producto', 'Color', 'Subtotal Bruto']
Target=['Cantidad']

X = data[Feautures]
y = data[Target]

# Identify categorical columns
object_columns = X.select_dtypes(include=['object']).columns

# Apply one-hot encoder to each column
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), object_columns)],
    remainder='passthrough'
)

X_processed = preprocessor.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=1)

# Train a random forest regressor
rf = RandomForestRegressor(n_estimators=100, random_state=1)
rf.fit(X_train, y_train)
y_predrf = rf.predict(X_test)
    
mse = mean_squared_error(y_test, y_predrf)
r2 = r2_score(y_test, y_predrf)
print(f"n_estimators = {100}: MSE = {mse:.2f}, R² = {r2:.4f}")


n_estimators = 100: MSE = 0.01, R² = 0.9981


In [ ]:
test_data['Month_sin'] = np.sin(2 * np.pi * test_data['Month'] / 12)
test_data['Month_cos'] = np.cos(2 * np.pi * test_data['Month'] / 12)


X_test_excel = test_data[Feautures]


X_test_processed = preprocessor.transform(X_test_excel)


predictions = rf.predict(X_test_processed)


test_data['Predicted_Cantidad'] = predictions


test_data.to_excel("predicciones.xlsx", index=False)

print("Predicciones guardadas en 'predicciones.xlsx'")

KeyError: 'Month'